<a href="https://colab.research.google.com/github/Tejaswini170104/CH5020-Term-paper-presentation/blob/main/CartPole_v1_sarsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import gymnasium as gym
import random
import wandb

# Constants
GAMMA = 0.99
EPSILON_START = 1
EPSILON_MIN = 0.01
MAX_EPISODES = 10000
MAX_STEPS = 500
NUM_RUNS = 5  # Running the experiment 5 times
MAX_REWARD = 500  # Max possible reward in CartPole-v1

# State bounds for normalization
state_bounds = [
    [-4.8, 4.8],  # Cart position
    [-3.0, 3.0],  # Cart velocity
    [-0.418, 0.418],  # Pole angle
    [-3.0, 3.0]   # Pole angular velocity
]
NUM_BINS = [20, 20, 20, 20]  # Number of bins for each state variable

def discretize_state(state):
    """Convert continuous state into discrete bins."""
    bin_indices = []
    for i in range(len(state)):
        bin_width = (state_bounds[i][1] - state_bounds[i][0]) / NUM_BINS[i]
        bin_index = int((state[i] - state_bounds[i][0]) / bin_width)
        bin_index = min(NUM_BINS[i] - 1, max(0, bin_index))  # Keep within bounds
        bin_indices.append(bin_index)
    return tuple(bin_indices)

def choose_action(q_table, state, epsilon):
    """Epsilon-greedy action selection."""
    if random.uniform(0, 1) < epsilon:
        return random.choice([0, 1])  # Random action
    return np.argmax(q_table.get(state, [0, 0]))  # Exploit best known action

def sarsa_q_table(env, alpha, lambda_decay, run_num):
    """SARSA algorithm with discretized states."""
    q_table = {}  # Q-table stored as a dictionary
    rewards_per_episode = []
    regret_per_episode = []

    for episode in range(MAX_EPISODES):
        epsilon = max(EPSILON_MIN, EPSILON_START * np.exp(-lambda_decay * episode))
        state = discretize_state(env.reset()[0])
        action = choose_action(q_table, state, epsilon)
        total_reward = 0

        for _ in range(MAX_STEPS):
            next_state_raw, reward, terminated, truncated, _ = env.step(action)
            next_state = discretize_state(next_state_raw)
            next_action = choose_action(q_table, next_state, epsilon)

            td_target = reward + (GAMMA * q_table.get(next_state, [0, 0])[next_action] if not (terminated or truncated) else 0)
            q_table.setdefault(state, [0, 0])[action] += alpha * (td_target - q_table[state][action])

            state, action = next_state, next_action
            total_reward += reward
            if terminated or truncated:
                break

        rewards_per_episode.append(total_reward)
        regret_per_episode.append(MAX_REWARD - total_reward)  # Compute regret

    return rewards_per_episode, regret_per_episode

# --------------------- #
#       W&B SWEEP      #
# --------------------- #

# Define the WandB sweep configuration
sweep_config = {
    "method": "grid",  # Options: "grid", "random", "bayes"
    "metric": {"name": "total_regret", "goal": "minimize"},
    "parameters": {
        "alpha": {"values": [0.1, 0.05, 0.01]},  # Learning rates
        "lambda_decay": {"values": [0.001, 0.005, 0.0005]}  # Decay rates
    }
}

# Initialize sweep
sweep_id = wandb.sweep(sweep_config, project="RL CartPole SARSA")

def train_sweep():
    """Train SARSA for a single (alpha, lambda_decay) configuration."""
    wandb.init(project="cpsarsa")  # Initialize WandB
    config = wandb.config  # Get current hyperparameters

    alpha = config.alpha
    lambda_decay = config.lambda_decay

    all_rewards = np.zeros((NUM_RUNS, MAX_EPISODES))
    all_regrets = np.zeros((NUM_RUNS, MAX_EPISODES))

    for run in range(NUM_RUNS):
        env = gym.make("CartPole-v1")
        rewards, regrets = sarsa_q_table(env, alpha, lambda_decay, run)
        all_rewards[run, :] = rewards
        all_regrets[run, :] = regrets

    # Compute mean and variance for rewards & regret per episode
    mean_rewards = np.mean(all_rewards, axis=0)
    var_rewards = np.var(all_rewards, axis=0)
    mean_regret = np.mean(all_regrets, axis=0)
    var_regret = np.var(all_regrets, axis=0)

    # Log episode-wise metrics
    for episode in range(MAX_EPISODES):
        wandb.log({
            "episode": episode,
            f"mean_reward_alpha_{alpha}_lambda_{lambda_decay}": mean_rewards[episode],
            f"reward_variance_alpha_{alpha}_lambda_{lambda_decay}": var_rewards[episode],
            f"mean_regret_alpha_{alpha}_lambda_{lambda_decay}": mean_regret[episode],
            f"regret_variance_alpha_{alpha}_lambda_{lambda_decay}": var_regret[episode]
        })

    # Log final performance
    wandb.log({
        f"final_mean_reward_alpha_{alpha}_lambda_{lambda_decay}": mean_rewards[-1],
        f"final_reward_variance_alpha_{alpha}_lambda_{lambda_decay}": var_rewards[-1],
        f"final_total_regret_alpha_{alpha}_lambda_{lambda_decay}": mean_regret[-1],
        f"final_regret_variance_alpha_{alpha}_lambda_{lambda_decay}": var_regret[-1]
    })

# Run the sweep agent for all 9 configurations
wandb.agent(sweep_id, train_sweep)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

wandb: Agent Starting Run: sl3ts817 with config:
wandb: 	alpha: 0.01
wandb: 	lambda_decay: 0.005


episode,▁▁▁▁▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇████
final_mean_reward_alpha_0.01_lambda_0.005,▁
final_regret_variance_alpha_0.01_lambda_0.005,▁
final_reward_variance_alpha_0.01_lambda_0.005,▁
final_total_regret_alpha_0.01_lambda_0.005,▁
mean_regret_alpha_0.01_lambda_0.005,▇▇▆▇▇▇▆▆▁▅▇▆▇▃▆▄█▇█▇█▄▅▇▅▆▇▄█▇▃▆▇▅▄▄▅▆▇▃
mean_reward_alpha_0.01_lambda_0.005,▂▁▁▂▁▃▃▃▂▂▂▃█▃▁▃▃▂▂▅▃▅▂▂▄▅▇▃▂▂▂▅▂▆▂▂▄▃▂▂
regret_variance_alpha_0.01_lambda_0.005,▁▁▁▁▁▂▂▁▁▁▁▁▂▃▁▁▁▂▃▁▂▂▁▁▂▁▁▁▁█▁▁▂▂▁▁▁▃▁▅
reward_variance_alpha_0.01_lambda_0.005,▁▁▁▃▁▁▂▂▃▁▁▁▁▁▁▁▁▁▂▃▁▃▁▂▃▁▁▂▇▁▁▂▆▁▁▇▃▁▆█
episode,9999
final_mean_reward_alpha_0.01_lambda_0.005,38.4


wandb: Agent Starting Run: g0ym3u7g with config:
wandb: 	alpha: 0.01
wandb: 	lambda_decay: 0.0005


episode,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
final_mean_reward_alpha_0.01_lambda_0.0005,▁
final_regret_variance_alpha_0.01_lambda_0.0005,▁
final_reward_variance_alpha_0.01_lambda_0.0005,▁
final_total_regret_alpha_0.01_lambda_0.0005,▁
mean_regret_alpha_0.01_lambda_0.0005,▇███▇▇█▆▇▇▆▆▇█▆▆▆▆▇▄▅▄▄▆▆▅▅▄▂▃▄▆▆▁▆▆▅▄▅▅
mean_reward_alpha_0.01_lambda_0.0005,▁▁▁▂▁▁▁▃▁▂▂▄▂▃▂▄▃▃▃▄▃▄▄▆▆▅▃▅▆▃█▄▄▄▃▅▅▇▄▅
regret_variance_alpha_0.01_lambda_0.0005,▁▁▁▁▂▁▁▁▁▂▁▁▄▁▁▂▁▆▃▄▃▃▆▄▂▃▂▂▄▅▂▃█▂▅▄▂▄▃▄
reward_variance_alpha_0.01_lambda_0.0005,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▂▁█▁
episode,9999
final_mean_reward_alpha_0.01_lambda_0.0005,91.2


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
